In [303]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [304]:
n_customers = 100
n_products = 10
n_days = 180
product_names = ['Avocado','Banana','Beef','Cauliflower','Egg','Milk','Shrimp','Tomato','Yogurt','Zucchini']

end_date = datetime.today() + timedelta(days=14)
start_date = end_date - timedelta(days=n_days-1)  # Approximate 6 months as 180 days
dates = pd.date_range(start=start_date, end=end_date, freq='D')

#### stage one model - quantity prediction

In [305]:
def stage1_pred(data, input_cols, prods, output_dir):
    #input_cols = ['OrderQuantity_lag1','Temperature', 'Rainfall', 'CPI', 'Female','Age','Income',]
    stage1_models = {}
    tmp = []
    for prod in prods:
        data_tmp = data[data['ProductID']==prod]
        X = data_tmp[input_cols]
        X = sm.add_constant(X)
        y = data_tmp['OrderQuantity']
        model = sm.OLS(y, X).fit()
        stage1_models[prod]=model
        predictions = model.predict(X)
        data_tmp['pred_s1'] = predictions
        tmp.append(data_tmp)
    data = pd.concat(tmp)
    data.to_csv(output_dir, index=False)
    return stage1_models, data

data = pd.read_csv('./data/stage1_data.csv')
## drop rows with missing records
data = data.dropna()
input_cols = ['OrderQuantity_lag1','Temperature', 'Rainfall', 'CPI', 'MCPI', 'Female','Age','Income',]
stage1_models, data = stage1_pred(data, input_cols, [i+1 for i in range(n_products)], './data/stage1_model_data.csv')

In [306]:
for prod in [i+1 for i in range(n_products)]:
    print(stage1_models[prod].summary())

                            OLS Regression Results                            
Dep. Variable:          OrderQuantity   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.408
Method:                 Least Squares   F-statistic:                     1761.
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        09:25:07   Log-Likelihood:                -47451.
No. Observations:               17900   AIC:                         9.492e+04
Df Residuals:                   17892   BIC:                         9.498e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                -19.6065      3

In [307]:
data.head()

,CustomerID,ProductID,Date,OrderQuantity,OrderQuantity_lag1,Day,Product,Temperature,Rainfall,CPI,MCPI,Female,Age,Income,intercept,pred_s1
1,1,1,2024-01-06,6,6.0,2,Avocado,42.22,0.00,100.7,100.7,0,60,59662.22,1,6.704962
2,1,1,2024-01-07,2,6.0,3,Avocado,45.34,0.00,100.7,100.7,0,60,59662.22,1,6.627071
3,1,1,2024-01-08,6,2.0,4,Avocado,51.87,0.00,100.7,100.7,0,60,59662.22,1,5.078323
4,1,1,2024-01-09,0,6.0,5,Avocado,50.23,62.52,100.7,100.7,0,60,59662.22,1,4.021067
5,1,1,2024-01-10,3,0.0,6,Avocado,36.23,0.00,100.7,100.7,0,60,59662.22,1,4.775914


#### stage two model - membership likehood prediction

In [308]:
def stage2_pred(data, input_cols, yvar):

    X = data[input_cols]
    y = data[yvar]
    logit_model = sm.Logit(y, X)
    model = logit_model.fit()
    predictions = model.predict(X)
    data['pred_s2'] = predictions
    return model, data

data = pd.read_csv('./data/stage2_data.csv')
print(data.columns)
## drop rows with missing records
input_cols = ['p'+str(i+1)+'_m' for i in range(n_products)]+['Temperature', 'Rainfall', 'CPI', 'MCPI', 'Female','Age','Income',]
stage2_model, data = stage2_pred(data, input_cols, 'Membership_Choice')

Index(['CustomerID', 'Date', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8',
       'p9', 'p10', 'Temperature', 'Rainfall', 'CPI', 'MCPI', 'Female', 'Age',
       'Income', 'p1_m', 'p2_m', 'p3_m', 'p4_m', 'p5_m', 'p6_m', 'p7_m',
       'p8_m', 'p9_m', 'p10_m', 'logit_t', 'Membership_Choice'],
      dtype='object')
Optimization terminated successfully.
         Current function value: 0.131525
         Iterations 12


In [309]:
stage2_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:      Membership_Choice   No. Observations:                17900
Model:                          Logit   Df Residuals:                    17884
Method:                           MLE   Df Model:                           15
Date:                Wed, 19 Jun 2024   Pseudo R-squ.:                  0.7993
Time:                        09:25:07   Log-Likelihood:                -2354.3
converged:                       True   LL-Null:                       -11729.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
p1_m            0.0637        nan        nan        nan         nan         nan
p2_m            0.1688      0.024      6.999      0.000       0.122       0.216
p3_m            0.0958      0.024      3.979      0.000       0.049       0.143
p4_m            0.1402      0.015      9.316      0.000       0.111       0.170
p5_m            0.3520      0.024     14.393      0.000       0.304       0.400
p6_m            0.1667      0.036      4.583      0.000       0.095       0.238
p7_m            0.1064      0.022      4.853      0.000       0.063       0.149
p8_m            0.3791      0.040      9.506      0.000       0.301       0.457
p9_m            0.0570      0.019      3.035      0.002       0.020       0.094
p10_m           0.1496      0.015      9.981      0.000       0.120       0.179
Temperature     0.0705      0.002     33.940      0.000       0.066       0.075
Rainfall       -0.3865      0.011    -34.236      0.000      -0.409      -0.364
CPI            -0.0918   1.37e+12  -6.72e-14      1.000   -2.68e+12    2.68e+12
MCPI           -0.0920   1.37e+12  -6.73e-14      1.000   -2.68e+12    2.68e+12
Female         -1.8046      0.081    -22.380      0.000      -1.963      -1.647
Age             0.0320      0.004      8.217      0.000       0.024       0.040
Income       5.543e-05   2.38e-06     23.274      0.000    5.08e-05    6.01e-05
===============================================================================

Possibly complete quasi-separation: A fraction 0.43 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [310]:
data.Membership_Choice.describe()

count    17900.000000
mean         0.363184
std          0.480931
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: Membership_Choice, dtype: float64

#### stage three model - seller prediction

In [311]:
def stage3_pred(data, dep_indep):
    for i in list(dep_indep.keys()):
        X = data[dep_indep[i]]
        X = sm.add_constant(X)
        y = data[[i]]
        model = sm.OLS(y, X).fit()
        predictions = model.predict(X)
        data[f'pred_s3_p{i[3:]}'] = predictions
        print(model.summary())
    return data

data = pd.read_csv('./data/stage3_data.csv')

dep_indep = {f'a_p{i}': [f'cp_p{i}', f'price_market_p{i}'] for i in range(1, 11)}

data = stage3_pred(data, dep_indep)

                            OLS Regression Results                            
Dep. Variable:                   a_p1   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     9625.
Date:                Wed, 19 Jun 2024   Prob (F-statistic):          2.64e-181
Time:                        09:25:07   Log-Likelihood:                 281.07
No. Observations:                 180   AIC:                            -556.1
Df Residuals:                     177   BIC:                            -546.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.4938      0.006     

In [312]:
data

,Date,cp_p1,price_market_p1,cp_p2,price_market_p2,cp_p3,price_market_p3,cp_p4,price_market_p4,cp_p5,...,pred_s3_p1,pred_s3_p2,pred_s3_p3,pred_s3_p4,pred_s3_p5,pred_s3_p6,pred_s3_p7,pred_s3_p8,pred_s3_p9,pred_s3_p10
0,2024-01-06,5.528105,1.460061,5.396914,12.134800,5.226258,0.382673,7.197404,21.315740,10.294869,...,1.135051,2.795292,2.125083,7.639403,10.086909,15.312922,23.248791,21.818076,34.161306,38.601221
1,2024-01-07,2.800314,-8.749513,4.007542,-2.085737,4.979415,8.165602,7.366689,5.944400,8.045070,...,0.345399,1.093405,3.220516,4.637097,7.980641,16.367607,21.593655,20.981075,29.170912,48.067939
2,2024-01-08,3.957476,-1.436184,5.863697,7.579802,6.367851,18.675449,9.193813,16.502178,11.758780,...,0.829103,2.434396,5.209190,7.480027,10.631882,17.943489,16.153683,25.037412,32.292957,46.089753
3,2024-01-09,6.481786,-17.234032,4.679930,25.182612,5.229020,16.903116,6.025427,40.234719,11.270849,...,0.300873,3.954380,4.606151,10.947472,16.249801,15.960139,27.148372,29.498087,30.176747,41.615400
4,2024-01-10,5.735116,11.252315,3.968636,6.153546,2.796328,6.482708,7.197531,25.053869,11.085222,...,1.644667,1.908952,2.321932,8.386119,12.134365,14.385216,22.281335,27.609590,31.979787,50.910662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2024-06-29,3.363189,-0.211893,2.829138,8.987185,9.995912,24.387469,5.473654,29.361076,8.871364,...,0.829043,1.961895,7.140211,8.553850,9.799670,11.843549,17.423130,25.629624,36.544535,42.287193
176,2024-06-30,0.393181,-13.430696,3.476709,1.968586,4.286901,21.071117,2.445282,28.091803,10.454692,...,-0.135584,1.391323,4.953150,7.083607,12.042354,14.729515,18.389656,32.618577,37.835207,48.957987
177,2024-07-01,0.620900,0.220260,3.635510,5.356623,2.916825,4.518296,10.303468,8.019071,13.228499,...,0.568589,1.762000,2.062663,6.231413,13.811585,15.043759,23.172415,25.455121,35.376555,36.092741
178,2024-07-02,1.088935,0.203442,3.594206,20.217906,11.188849,6.397375,6.821542,24.066571,12.017195,...,0.615881,3.238845,4.792281,8.037852,11.575044,13.401224,20.049258,30.067383,30.298012,49.588601


In [313]:

n_products = 10
stage3_data_long = pd.DataFrame()
for prod_id in range(1,n_products+1):
    tmp = data[['Date', f'cp_p{prod_id}',f'price_market_p{prod_id}', f'a_p{prod_id}', f'pred_s3_p{prod_id}']]
    #print(tmp.shape)
    tmp['ProductID']= prod_id
    tmp.columns = ['Date', 'cp','price_market', 'a', 'pred_s3', 'ProductID']
    #print(tmp.shape)
    stage3_data_long = pd.concat([stage3_data_long, tmp])
stage3_data_long.reset_index(inplace=True)
stage3_data_long.head(2)

,index,Date,cp,price_market,a,pred_s3,ProductID
0,0,2024-01-06,5.528105,1.460061,1.152153,1.135051,1
1,1,2024-01-07,2.800314,-8.749513,0.347609,0.345399,1


In [314]:
stage3_data_long['Demand_Priority'] = stage3_data_long.groupby(['Date'])['a'].rank(ascending=False)
stage3_data_long['Profitability_Priority'] = stage3_data_long.groupby(['Date'])['cp'].rank(ascending=False)

In [315]:
stage3_data_long['ProductID'] = stage3_data_long['ProductID'].astype(int)
stage3_data_long.to_csv('./data/stage3_data_store.csv', index=False)

In [319]:
stage1_data_store = pd.read_csv('./data/stage1_data_store.csv')
stage2_data_store = pd.read_csv('./data/stage2_data_store.csv')
df = stage3_data_long.merge(stage1_data_store, on=['Date', 'ProductID'], how='inner')
df = df.merge(stage2_data_store, on='Date', how='inner')
df.sort_values(by=['Date','Low_Stock_Priority', 'Demand_Priority', 'Profitability_Priority'], ascending=True, inplace=True)

product_map = {i+1: name for i, name in enumerate(product_names)}
df['Product'] = df['ProductID'].map(product_map)

In [322]:
df = df[['Date', 'ProductID', 'Product', 'Low_Stock_Priority', 'Demand_Priority', 'Profitability_Priority', 'Low_Stock_Risk', 'cp', 'price_market', 'a', 'pred_s3',
        'OrderQuantity', 'Inventory', 'Temperature', 'Rainfall', 'CPI', 'MCPI', 'Female', 'Age', 'Income']]

df.to_csv('./data/data_demo.csv')

In [323]:
df.tail(10)

,Date,ProductID,Product,Low_Stock_Priority,Demand_Priority,Profitability_Priority,Low_Stock_Risk,cp,price_market,a,pred_s3,OrderQuantity,Inventory,Temperature,Rainfall,CPI,MCPI,Female,Age,Income
1073,2024-07-02,6,Milk,1,5.0,7.0,-2.347826,10.382565,22.268075,13.357239,13.401224,77,23,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
894,2024-07-02,5,Egg,1,6.0,5.0,-0.422764,12.017195,20.292575,11.529019,11.575044,350,246,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
536,2024-07-02,3,Beef,1,8.0,6.0,-0.436464,11.188849,6.397375,4.763539,4.792281,520,362,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
178,2024-07-02,1,Avocado,1,10.0,10.0,-0.397727,1.088935,0.203442,0.566343,0.615881,123,88,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
1610,2024-07-02,9,Yogurt,2,2.0,3.0,-0.109785,15.781855,34.652047,30.244368,30.298012,465,419,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
1789,2024-07-02,10,Zucchini,5,1.0,1.0,0.032445,23.093344,51.983271,49.532257,49.588601,835,863,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
1252,2024-07-02,7,Shrimp,5,4.0,4.0,0.009585,13.053278,29.752580,19.997976,20.049258,310,313,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
715,2024-07-02,4,Cauliflower,5,7.0,8.0,0.001377,6.821542,24.066571,7.989209,8.037852,725,726,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
1431,2024-07-02,8,Tomato,6,3.0,2.0,0.078838,18.608428,36.672412,30.002985,30.067383,222,241,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
357,2024-07-02,2,Banana,8,9.0,9.0,0.194888,3.594206,20.217906,3.187910,3.238845,252,313,80.09,62.04,105.16,105.16,0.56,44.08,49946.1506
